# 03b - Vertex AI + BQML - Online Predictions with BQML Models

Models built with BigQuery ML (BQML), like the one in (03a), can also be exported for use and deployment outside of BigQuery.  A Vertex AI Endpoint can be used for online predictions with an exported model.  This demonstration shows the process of exporting and deploying a BQML model with Vertex AI.

**Prerequisites:**

-  03a - BigQuery Machine Learning (BQML) - Machine Learning with SQL

**Overview:**

-  Export the BigQuery ML model built in (03a) to a GCS bucket URI
   -  EXPORT MODEL …
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Upload Model
      -  Model - aiplatform.Model.upoad
   -  Create Endpoint
      -  Endpoint - aiplatform.Endpoint.create
   -  Deploy to Endpoint
      -  Endpoint.deploy(model=Model)
   -  Online Predictions
      -  Endpoint.predict

**Resources:**

-  [Export formats for BigQuery ML models](https://cloud.google.com/bigquery-ml/docs/exporting-models)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)

**Related Training:**

-  todo

---
## Conceptual Architecture

<img src="architectures/statmike-mlops-03.png">

---
## Setup

inputs:

In [1]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'digits'
NOTEBOOK = '03b'

# Resources
DEPLOY_IMAGE='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'target'
VAR_OMIT = 'target_OE' # add more variables to the string with space delimiters

packages:

In [2]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [4]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
params = {"URI": URI}
DIR = f"temp/{NOTEBOOK}"

environment:

In [6]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Export the BigQuery Model

Export the BigQuery Model:
- https://cloud.google.com/bigquery-ml/docs/exporting-models

In [13]:
export = bigquery.query(query = f"EXPORT MODEL {DATANAME}.{DATANAME}_lr OPTIONS(URI = '{URI}')")

In [16]:
export.result()

---
## Serving

### Upload The Model

In [13]:
model = aiplatform.Model.upload(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = URI,
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/691911073727/locations/us-central1/models/6536614219314364416/operations/6126991987020136448
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/691911073727/locations/us-central1/models/6536614219314364416
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/691911073727/locations/us-central1/models/6536614219314364416')


In [6]:
model.display_name

'MODEL_03_BQML-DIGITS'

### Create An Endpoint

In [14]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/691911073727/locations/us-central1/endpoints/5021777467308769280/operations/983881212563030016
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/691911073727/locations/us-central1/endpoints/5021777467308769280
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/691911073727/locations/us-central1/endpoints/5021777467308769280')


In [8]:
endpoint.display_name

'ENDPOINT_03_BQML-DIGITS'

### Deploy Model To Endpoint

In [15]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/691911073727/locations/us-central1/models/6536614219314364416 to Endpoint : projects/691911073727/locations/us-central1/endpoints/5021777467308769280
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/691911073727/locations/us-central1/endpoints/5021777467308769280/operations/5325351253348188160
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/691911073727/locations/us-central1/endpoints/5021777467308769280


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [17]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME} LIMIT 10").to_dataframe()

In [18]:
pred.head(4)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p56,p57,p58,p59,p60,p61,p62,p63,target,target_OE
0,0.0,5.0,16.0,15.0,5.0,0.0,0.0,0.0,0.0,2.0,...,0.0,6.0,16.0,16.0,16.0,16.0,7.0,0.0,2,Even
1,0.0,5.0,16.0,12.0,1.0,0.0,0.0,0.0,0.0,5.0,...,0.0,8.0,16.0,16.0,16.0,16.0,4.0,0.0,2,Even
2,0.0,5.0,15.0,16.0,6.0,0.0,0.0,0.0,0.0,11.0,...,0.0,6.0,16.0,16.0,16.0,13.0,3.0,0.0,2,Even
3,0.0,4.0,15.0,15.0,8.0,0.0,0.0,0.0,0.0,8.0,...,0.0,7.0,14.0,11.0,0.0,0.0,0.0,0.0,2,Even


In [23]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET])]].to_dict(orient='records')[0]
#newob

In [24]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [25]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[{'target_probs': [0.001038740586702898, 0.9965215668681598, 0.001404794612669977, 2.832550369749307e-06, 2.568335928814314e-05, 0.0001207048328886546, 1.270623994621032e-05, 0.0002614810419893859, 0.0005984386745938329, 1.305123339155052e-05], 'target_values': ['1', '2', '3', '4', '9', '6', '0', '5', '8', '7'], 'predicted_target': ['2']}], deployed_model_id='5481522861300514816', explanations=None)

In [26]:
prediction.predictions[0]#['classes'][np.argmax(prediction.predictions[0]['scores'])]

{'target_probs': [0.001038740586702898,
  0.9965215668681598,
  0.001404794612669977,
  2.832550369749307e-06,
  2.568335928814314e-05,
  0.0001207048328886546,
  1.270623994621032e-05,
  0.0002614810419893859,
  0.0005984386745938329,
  1.305123339155052e-05],
 'target_values': ['1', '2', '3', '4', '9', '6', '0', '5', '8', '7'],
 'predicted_target': ['2']}

In [29]:
prediction.predictions[0]['target_values'][np.argmax(prediction.predictions[0]['target_probs'])]

'2'

### Get Predictions: REST

In [76]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [77]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    {
      "scores": [
        0.00097372796153649688,
        0.001646753866225481,
        0.010825014673173429,
        0.001175398472696543,
        0.00073917064582929015,
        3.5786611078947321e-06,
        0.0034829874057322741,
        0.017448257654905319,
        0.96323353052139282,
        0.00047153508057817822
      ],
      "classes": [
        "1",
        "9",
        "7",
        "4",
        "6",
        "5",
        "3",
        "8",
        "2",
        "0"
      ]
    }
  ],
  "deployedModelId": "2102697240865800192"
}


### Get Predictions: gcloud (CLI)

In [78]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[{'classes': ['1', '9', '7', '4', '6', '5', '3', '8', '2', '0'], 'scores': [0.0009737279615364969, 0.001646753866225481, 0.01082501467317343, 0.001175398472696543, 0.0007391706458292902, 3.578661107894732e-06, 0.003482987405732274, 0.01744825765490532, 0.9632335305213928, 0.0004715350805781782]}]


---
## Remove Resources
see notebook "XX - Cleanup"